In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embedder = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embedder, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            Before answering, revise what the question was. Then answer the question.
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    context = "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )
    return context

map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough()
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            The following extracted parts have all information about the question.
            Given the following extracted parts of a long document and a question, create a final answer.
            When making the answer, use the information as much as possible.
            If you don't know the answer, just say that you don't know. Don't try to make up an answer. And tell the reason why you don't know the answer
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = {"context": map_chain, "question": RunnablePassthrough(history=load_memory)} | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result)


In [2]:
invoke_chain("Is Aaronson guilty?")


content='Based on the text provided, Aaronson is considered guilty of the crimes he was charged with.'


In [6]:
invoke_chain("What message did he write in the table?")


content='He wrote "2+2=5" in the dust on the table.'


In [4]:
invoke_chain("Who is Julia?")


content='Julia is a character mentioned in the text provided. She is someone who was with the main character in the Ministry of Love and was part of his attempt to resist the power of the Party. There are hints of a romantic or personal relationship between Julia and the protagonist.'
